# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.8 which is incompatible.


The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np
from collections import deque
import torch
import time

# select this option to load version 1 (with a single agent) of the environment
#env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
def train_agent(start_episode=1, end_episode=2000, max_t=750, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    training_start = time.time()
    scores = []                        # list containing scores from each episode
    episode_seconds = []
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    training_time = 0
    for i_episode in range(start_episode, end_episode+1):
        episode_start = time.time()
        agent.reset()                                      # reset the agent for noise
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        state = env_info.vector_observations               # get the current state
        score = np.zeros(num_agents)
        for t in range(max_t):
            action = agent.act(state, eps)                 # select an action
            action = np.clip(action, -1, 1)                # all action values are between -1 and 1
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations      # get the next state
            reward = env_info.rewards                      # get the reward
            done = env_info.local_done                     # see if episode has finished
            score += reward                                # update the score
            agent.step(state, action, reward, next_state, done)
            state = next_state                             # roll over the state to next time step
            if np.any(done):                               # exit loop if episode finished
                break
        episode_elapsed = time.time() - episode_start
        episode_seconds.append(episode_elapsed)
        training_time += episode_elapsed
        scores_window.append(np.mean(score))       # save most recent score
        scores.append(np.mean(score))              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        elapsed_str = time.strftime("%H:%M:%S", time.gmtime(time.time() - training_start))
        print('\rEpisode {:04} Elapsed: {} Average Score: {:.2f}'.format(i_episode, 
                                                                          elapsed_str, np.mean(scores_window)), end="")
        if i_episode % 10 == 0:
            print('\rEpisode {:04} Elapsed: {} Average Score: {:.2f}'.format(i_episode, 
                                                                              elapsed_str, np.mean(scores_window)))
        if i_episode % 10 == 0:
            actor_state = {'epoch': i_episode, 'state_dict': agent.actor_local.state_dict(),
                     'optimizer': agent.actor_optimizer.state_dict(),}
            torch.save(actor_state, 'ddpg_checkpoint_actor_{:0>4d}.pth'.format(i_episode))
            critic_state = {'epoch': i_episode, 'state_dict': agent.critic_local.state_dict(),
                     'optimizer': agent.critic_optimizer.state_dict(),}
            torch.save(critic_state, 'ddpg_checkpoint_critic_{:0>4d}.pth'.format(i_episode))
            np.save('ddpg_results_{:0>4d}'.format(i_episode), scores)
            np.save('ddpg_times_{:0>4d}'.format(i_episode), episode_seconds)
        if np.mean(scores_window)>=30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.actor_local.state_dict(), 'ddpg_checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'ddpg_checkpoint_critic.pth')
            np.save('ddpg_solved_results', scores)
            np.save('ddpg_solved_times', episode_seconds)
            break
    return scores, episode_seconds

In [6]:
from ddpg_agent import DdpgAgent

agent = DdpgAgent(state_size=state_size, action_size=action_size, random_seed=7, num_agents=20)
start_episode=1
end_episode=1000

action_size: 4


In [7]:
# restore our saved model
#actor_state = torch.load('final_checkpoint_actor_0500.pth')
#agent.actor_local.load_state_dict(actor_state['state_dict'])
#agent.actor_optimizer.load_state_dict(actor_state['optimizer'])
#critic_state = torch.load('final_checkpoint_critic_0500.pth')
#agent.critic_local.load_state_dict(critic_state['state_dict'])
#agent.critic_optimizer.load_state_dict(critic_state['optimizer'])
#start_episode=501

In [8]:
from workspace_utils import active_session

with active_session():
    ddpg_scores, episode_seconds = train_agent(start_episode=start_episode, end_episode=end_episode)

Episode 0010 Elapsed: 00:06:54 Average Score: 0.27
Episode 0020 Elapsed: 00:17:43 Average Score: 0.26
Episode 0030 Elapsed: 00:32:19 Average Score: 0.26
Episode 0040 Elapsed: 00:50:44 Average Score: 0.26
Episode 0050 Elapsed: 01:12:56 Average Score: 0.25
Episode 0060 Elapsed: 01:39:09 Average Score: 0.26
Episode 0070 Elapsed: 02:08:59 Average Score: 0.26
Episode 0080 Elapsed: 02:39:51 Average Score: 0.25
Episode 0090 Elapsed: 03:10:37 Average Score: 0.25
Episode 0100 Elapsed: 03:41:22 Average Score: 0.25
Episode 0110 Elapsed: 04:12:10 Average Score: 0.25
Episode 0120 Elapsed: 04:43:04 Average Score: 0.25
Episode 0128 Elapsed: 05:07:50 Average Score: 0.25

KeyboardInterrupt: 

When finished, you can close the environment.

In [ ]:
env.close()

In [ ]:
%%javascript
Jupyter.notebook.session.delete();